In [13]:
# Importing Modules 

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import pickle
import json
import random

import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

words = []
classes = []
documents = []
ignore_words = ['!','?']
data = open('intents.json').read()
intents = json.loads(data)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [14]:
# Preprossing the data

for i in intents['intents']:
  for j in i['patterns']:

    w = nltk.word_tokenize(j)   #Tokeinzing each word 
    words.extend(w)

    documents.append((w, i['tag']))  #Adding documents 

    if i['tag'] not in classes:
      classes.append(i['tag'])    #Adding to the classes list

In [15]:
#Lemmatizing and discarding duplicate words 

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes))) #Sorting classes 

#print(len(documents),"Documents") #Combination between patterns and intents 
#print(len(classes), "Classes", classes) classes = intents 
#print(len(words), "Lematized Words", words) #all words, vocab

pickle.dump(words, open('Words.pkl','wb'))
pickle.dump(classes, open('Classes.pkl','wb'))

In [16]:
# Creating Training and Testing Data

training = []
out = [0] * len(classes) #empty array for output

for doc in documents:
  bag = []  #initilize bag of words 
  pattern_words = doc[0] #tokenized words for the pattern
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

  #If match found in current pattern create our bag of words array with 1
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    out_row = list(out)
    out_row[classes.index(doc[1])] = 1
    training.append([bag, out_row])

random.shuffle(training)
training = np.array(training)

X_train = list(training[:,0])
y_train = list(training[:,1])

In [ ]:
#Building the Model 

model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation = 'softmax'))

#Compling the Model

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

#Fitting and Saving the Model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs = 200, batch_size = 5, verbose = 1)
model.save('chat_model.h5',hist)
